### Connect to PostgreSQL database

In [1]:
import psycopg2
import pandas as pd

try:
    conn = psycopg2.connect(
        dbname="project", 
        user="postgres",       
        password="123",   
        host="localhost",          
        port="5432"                
    )
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")
    
cur = conn.cursor()

Connection successful!


## 1. Daily Sales by Store and Category

In [2]:
sql1 = """
SELECT s.name AS store, p.category, DATE(t.transaction_date) AS sale_date,
       SUM(ti.quantity * ti.unit_price) AS total_sales
FROM Sales_Transactions t
JOIN Transaction_Items ti ON t.transaction_id = ti.transaction_id
JOIN Products p ON ti.product_id = p.product_id
JOIN Stores s ON t.store_id = s.store_id
GROUP BY s.name, p.category, DATE(t.transaction_date)
ORDER BY sale_date DESC, store;
"""
df1 = pd.read_sql_query(sql1, conn)
df1.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\3525661512.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(sql1, conn)


,store,category,sale_date,total_sales
0,"Harris, Lloyd and Payne",Furniture,2025-08-07,1081.41
1,"Harris, Lloyd and Payne",Toys,2025-08-07,424.04
2,"King, Carlson and Ramos",Food,2025-08-07,769.72
3,"Todd, Mcdowell and Mclean",Electronics,2025-08-05,1740.50
4,"Todd, Mcdowell and Mclean",Food,2025-08-05,248.86


## 2. Top 10 Best-Selling Products (Last 30 Days)

In [3]:
category = None
sql2 = """
SELECT p.name, SUM(ti.quantity) AS units_sold
FROM Transaction_Items ti
JOIN Products p ON ti.product_id = p.product_id
JOIN Sales_Transactions t ON ti.transaction_id = t.transaction_id
WHERE t.transaction_date >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY p.name
ORDER BY units_sold DESC
LIMIT 10;
"""
df2 = pd.read_sql_query(sql2, conn)
df2.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\2414524911.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(sql2, conn)


,name,units_sold
0,Bacon,11
1,Cotton Chicken,6
2,Cotton Mouse,5
3,Small Hat,5
4,Soft Salad,5


## 3. Salesperson Performance Analysis

In [4]:
sql3 = """
SELECT e.first_name || ' ' || e.last_name AS employee,
       SUM(t.total_amount) AS total_sales,
       COUNT(DISTINCT t.transaction_id) AS transactions
FROM Sales_Transactions t
JOIN Employees e ON t.employee_id = e.employee_id
GROUP BY employee
ORDER BY total_sales DESC;
"""
df3 = pd.read_sql_query(sql3, conn)
df3.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\952030927.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3 = pd.read_sql_query(sql3, conn)


,employee,total_sales,transactions
0,Michelle Parks,16009.31,6
1,Alan Hayden,14026.31,8
2,Tracey Martinez,12421.42,5
3,Sarah Hernandez,11825.36,6
4,Michelle Campbell,11150.83,5


## 4. Customer Purchase Frequency and Recency (RFM Analysis)

In [5]:
sql4 = """
SELECT c.customer_id, c.first_name, c.last_name,
       MAX(t.transaction_date) AS last_purchase,
       COUNT(t.transaction_id) AS purchase_count,
       SUM(t.total_amount) AS total_spent
FROM Customers c
LEFT JOIN Sales_Transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id
ORDER BY last_purchase DESC;
"""
df4 = pd.read_sql_query(sql4, conn)
df4.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\105404546.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4 = pd.read_sql_query(sql4, conn)


,customer_id,first_name,last_name,last_purchase,purchase_count,total_spent
0,55,Paula,Sanchez,NaT,0,NaN
1,182384550,Bryan,Fowler,NaT,0,NaN
2,4,Matthew,Rojas,NaT,0,NaN
3,4332786,Jessica,Stafford,NaT,0,NaN
4,797,Eric,Buchanan,NaT,0,NaN


## 5. Product Affinity — Frequently Bought Together

In [6]:
sql5 = """
SELECT a.product_id AS product_a, b.product_id AS product_b,
       COUNT(*) AS times_bought_together
FROM Transaction_Items a
JOIN Transaction_Items b 
  ON a.transaction_id = b.transaction_id
 AND a.product_id < b.product_id
GROUP BY a.product_id, b.product_id
ORDER BY times_bought_together DESC
LIMIT 10;
"""
df5 = pd.read_sql_query(sql5, conn)
df5.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\380978017.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5 = pd.read_sql_query(sql5, conn)


,product_a,product_b,times_bought_together
0,73184,634587,3
1,941,634587,3
2,36097064,813731358,3
3,73184,36097064,3
4,159,36097064,3


## 6. Inventory Turnover Rate by Product

In [7]:
sql6 = """
SELECT p.name, 
       SUM(ti.quantity) / NULLIF(AVG(inv.quantity), 0) AS turnover_rate
FROM Transaction_Items ti
JOIN Products p ON ti.product_id = p.product_id
JOIN Inventory inv ON p.product_id = inv.product_id
GROUP BY p.name
ORDER BY turnover_rate DESC;
"""
df6 = pd.read_sql_query(sql6, conn)
df6.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\2112646571.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6 = pd.read_sql_query(sql6, conn)


,name,turnover_rate
0,Gorgeous Bike,NaN
1,Generic Shirt,11.654676
2,Cheese,10.596026
3,Small Hat,8.684825
4,Mouse,7.043478


## 7. Promotion Effectiveness

In [8]:
sql7 = """
SELECT pr.promo_id, p.name,
       SUM(CASE WHEN t.transaction_date BETWEEN pr.start_date AND pr.end_date 
                THEN ti.quantity ELSE 0 END) AS units_sold_during,
       SUM(CASE WHEN t.transaction_date NOT BETWEEN pr.start_date AND pr.end_date 
                THEN ti.quantity ELSE 0 END) AS units_sold_outside
FROM Promotions pr
JOIN Products p ON pr.product_id = p.product_id
JOIN Transaction_Items ti ON p.product_id = ti.product_id
JOIN Sales_Transactions t ON ti.transaction_id = t.transaction_id
GROUP BY pr.promo_id, p.name;
"""
df7 = pd.read_sql_query(sql7, conn)
df7.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\2454285733.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df7 = pd.read_sql_query(sql7, conn)


,promo_id,name,units_sold_during,units_sold_outside
0,8,Unbranded Granite Tuna,2,15
1,16,Used Concrete Pants,8,6
2,20,Soft Salad,10,8
3,2,Cotton Chicken,0,12
4,15,Generic Metal Tuna,0,9


## 8. On-Time Delivery Rate by Vendor

In [9]:
sql8 = """
SELECT v.name AS vendor,
       COUNT(*) FILTER (WHERE d.delivered_at <= o.order_date + INTERVAL '7 days') * 100.0 / COUNT(*) AS on_time_percentage
FROM Deliveries d
JOIN Orders o ON d.order_id = o.order_id
JOIN Vendors v ON o.vendor_id = v.vendor_id
GROUP BY v.name;
"""
df8 = pd.read_sql_query(sql8, conn)
df8.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\2645180439.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df8 = pd.read_sql_query(sql8, conn)


,vendor,on_time_percentage
0,Gibson Ltd,100.000000
1,Crawford LLC,100.000000
2,"Chan, Hale and Fisher",44.444444
3,Allen Ltd,100.000000
4,"Willis, York and Yates",50.000000


## 9. Store-Level Profitability Estimation

In [10]:
sql9 = """
SELECT s.name,
       SUM(ti.quantity * (ti.unit_price - p.unit_price * 0.6)) AS estimated_profit
FROM Transaction_Items ti
JOIN Products p ON ti.product_id = p.product_id
JOIN Sales_Transactions t ON ti.transaction_id = t.transaction_id
JOIN Stores s ON t.store_id = s.store_id
GROUP BY s.name;
"""
df9 = pd.read_sql_query(sql9, conn)
df9.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\1886012740.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df9 = pd.read_sql_query(sql9, conn)


,name,estimated_profit
0,"King, Carlson and Ramos",10040.924
1,"Harris, Lloyd and Payne",8865.884
2,"Todd, Mcdowell and Mclean",18543.484
3,Gill Group,7707.720
4,"Jones, Martinez and Lopez",10883.500


## 10. Shift Coverage Gaps

In [11]:
sql10 = """
SELECT store_id, 
       DATE(start_time) AS shift_date,
       COUNT(*) AS num_employees
FROM Shifts
GROUP BY store_id, DATE(start_time)
HAVING COUNT(*) < 3
ORDER BY shift_date;
"""
df10 = pd.read_sql_query(sql10, conn)
df10.head()

C:\Users\dengx\AppData\Local\Temp\ipykernel_438544\2108923852.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df10 = pd.read_sql_query(sql10, conn)


,store_id,shift_date,num_employees
0,1702891,2025-05-18,1
1,350,2025-05-19,1
2,1702891,2025-05-19,1
3,1702891,2025-05-20,1
4,7056020,2025-05-21,1
